In [ ]:
// Add required dependencies
%classpath add jar "C:/Users/Dell/Downloads/opencv/build/java/opencv-490.jar"
%classpath add jar C:/Users/Dell/Downloads/Tess4J-3.4.8-src/Tess4J/dist/tess4j-3.4.8.jar
//%classpath add jar /path/to/jna.jar
// Add the maven dependencies
%maven ai.djl:api:0.28.0
%maven ai.djl.pytorch:pytorch-model-zoo:0.29.0
%maven ai.djl.tensorflow:tensorflow-engine:0.28.0
%maven ai.djl.tensorflow:tensorflow-model-zoo:0.28.0
%maven ai.djl:basicdataset:0.28.0
%maven ai.djl.mxnet:mxnet-engine:0.28.0
%maven org.slf4j:slf4j-simple:1.7.36
%maven ai.djl:basicdataset:0.29.0
%maven ai.djl.pytorch:pytorch-engine:0.22.0
%maven ai.djl.pytorch:pytorch-model-zoo:0.22.0    
%maven ai.djl.opencv:opencv:0.22.0
%maven ai.djl.tensorflow:tensorflow-engine:0.22.0
%maven ai.djl.tensorflow:tensorflow-model-zoo:0.22.0
%maven net.sourceforge.tess4j:tess4j:4.5.4
%maven org.pytorch:pytorch:1.11.0
%maven org.pytorch:pytorch-native-cpu:1.10.0
// Load OpenCV native library
System.loadLibrary(org.opencv.core.Core.NATIVE_LIBRARY_NAME);

In [ ]:
import ai.djl.*;
import ai.djl.inference.*;
import ai.djl.ndarray.*;
import ai.djl.translate.*;
import java.nio.file.*;
import ai.djl.Model;
import ai.djl.ModelException;
import ai.djl.modality.Classifications;
import ai.djl.modality.cv.Image;
import ai.djl.modality.cv.ImageFactory;
import ai.djl.ndarray.NDManager;
import ai.djl.inference.Predictor;
import ai.djl.translate.TranslateException;
import net.sourceforge.tess4j.ITesseract;
import net.sourceforge.tess4j.Tesseract;
import net.sourceforge.tess4j.TesseractException;
import org.opencv.core.Core;
import org.opencv.core.Mat;
import org.opencv.core.Point;
import org.opencv.core.Rect;
import org.opencv.core.Scalar;
import org.opencv.core.Size;
import org.opencv.videoio.VideoCapture;
import org.opencv.imgproc.Imgproc;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;

public class YoloV8CarNumberRecognitionWithOCR {

    static { System.loadLibrary(Core.NATIVE_LIBRARY_NAME); }

    public static void main(String[] args) throws IOException, ModelException, TranslateException {
        String videoFile = "path/to/your/video.mp4";  // Path to your video file
        String yoloModelPath = "C:/Users/Dell/Desktop/licence_plate_task/runs/content/runs/detect/train2/weights/best.pt";  // Path to your YOLOv8 .pt model
        String outputFile = ""C:/Users/Dell/Desktop/licence_plate_task/results.txt"";

        // Initialize Tesseract OCR
        ITesseract tesseract = new Tesseract();
        tesseract.setDatapath("C:/Users/Dell/Downloads/Tess4J-3.4.8-src/Tess4J/tessdata"); // Set the path to the tessdata folder (containing language data files)
        
        try (Model model = Model.newInstance("YOLOv8", yoloModelPath)) {
            // YOLOv8 model loaded from the local file system
            model.load(new File(yoloModelPath).toPath());
            
            try (Predictor<Image, DetectedObjects> predictor = model.newPredictor()) {
                
                // Open video using OpenCV
                VideoCapture cap = new VideoCapture(videoFile);
                if (!cap.isOpened()) {
                    System.out.println("Error opening video file");
                    return;
                }

                // Prepare output file to write the car number detection results
                FileWriter writer = new FileWriter(outputFile);
                
                Mat frame = new Mat();
                int frameNumber = 0;
                
                // Loop through each frame in the video
                while (cap.read(frame)) {
                    frameNumber++;

                    // Convert OpenCV Mat to DJL Image
                    Image djlImage = OpenCVUtils.matToDJLImage(frame);

                    // Perform YOLOv8 detection
                    Classifications result = predictor.predict(djlImage);
                    
                    // Assuming the result contains the bounding boxes for license plates (requires custom translator if needed)
                    for (Classifications.Classification licensePlate : result.items()) {
                        Rect bbox = extractBoundingBoxFromResult(licensePlate);

                        // Crop the region of the detected license plate
                        Mat plateImage = new Mat(frame, bbox);
                        
                        // Perform OCR on the cropped license plate image
                        String plateText = performOcrOnImage(plateImage, tesseract);
                        
                        // Write the result to the output file
                        writer.write("Frame " + frameNumber + ": Detected plate - " + plateText + "\n");
                        
                        // Optionally: Draw the bounding box and text on the frame
                        Imgproc.rectangle(frame, bbox.tl(), bbox.br(), new Scalar(0, 255, 0), 2);
                        Imgproc.putText(frame, plateText, bbox.tl(), Imgproc.FONT_HERSHEY_SIMPLEX, 1, new Scalar(0, 255, 0), 2);
                    }

                    OpenCVUtils.displayFrame("Car Number Detection", frame);
                }

                cap.release();
                writer.close();
            }
        }
    }

    // Helper method to perform OCR on the cropped license plate image
    private static String performOcrOnImage(Mat plateImage, ITesseract tesseract) {
        String result = "";
        try {
            // Convert OpenCV Mat to buffered image for Tesseract
            BufferedImage bufferedImage = OpenCVUtils.matToBufferedImage(plateImage);
            result = tesseract.doOCR(bufferedImage);
        } catch (TesseractException e) {
            e.printStackTrace();
        }
        return result.trim();  // Trim any extra whitespaces
    }

    // Helper method to extract the bounding box of the detected license plate
    private static Rect extractBoundingBoxFromResult(Classifications.Classification licensePlate) {
        // Assuming result contains coordinates for bounding box (you need to adapt this based on your YOLOv8 output)
        double x = 0;  // Get X coordinate from licensePlate result
        double y = 0;  // Get Y coordinate from licensePlate result
        double width = 0;  // Get width from licensePlate result
        double height = 0;  // Get height from licensePlate result
        return new Rect(new Point(x, y), new Size(width, height));
    }
}
